In [ ]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from sklearn.cluster import DBSCAN

In [ ]:
#
#convert to numpy 2d array where each 3d value is an hsv coordinate
#mask out anything with an absorption value that corresponds to white (MAYBE EVEN TO LIGHT BLUE CELLS)
#kmeans clustering into 3-4 clusters in 3d
#calculate mean and variance of each hsv value. might be skewed towards light blue. Low pass filter for vibrance may be valuable
#take the 95% CI

In [3]:
file_path = r"C:\Users\yshen97\PycharmProjects\pythonProject\73__tile_00028.png"

# Check if the file exists
if not os.path.exists(file_path):
    print(f"File does not exist: {file_path}")
    print("Please check the file path and ensure it includes the file extension (e.g., .jpg, .png)")
else:
    # Try to read the image
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load the image: {file_path}")
        print("The file may be corrupted or in an unsupported format.")
    else:
        print(f"Image shape: {img.shape}")  # Print image dimensions
        cv2.imshow("Image", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
print("Original dimensions: ", img.shape)

Original dimensions:  (256, 256, 3)


In [4]:
hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# Reshape the array
flattened_array = hsv_image.reshape(-1, 3)

# Verify the new shape
print("Flattened array shape: ", flattened_array.shape)  # Should output (65536, 3)

# Create a boolean mask for values that don't meet the threshold in the third column
mask = flattened_array[:, 2] >= 210
# Set entire rows to 0 where the third column doesn't meet the threshold
flattened_array[mask] = 0

original_shape_array = flattened_array.reshape(256, 256, 3)
rgb_image = cv2.cvtColor(original_shape_array, cv2.COLOR_HSV2BGR)
cv2.imshow("Image", rgb_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Flattened array shape:  (65536, 3)


In [5]:
def create_binary_mask(image):
    # Ensure the image is 256x256x3
    assert image.shape == (256, 256, 3), "Image must be 256x256x3"

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a binary mask
    # Pixels with value 0 in all channels (black) will be 0, others will be 1
    mask = np.where(gray == 0, 0, 1).astype(np.uint8)

    return mask


# Create the binary mask
binary_mask = create_binary_mask(rgb_image)

# Display the original image and the mask
cv2.imshow('Original Image', rgb_image)
cv2.imshow('Binary Mask', binary_mask * 255)  # Multiply by 255 for better visualization
cv2.waitKey(0)
cv2.destroyAllWindows()

# If you want to save the mask
cv2.imwrite('binary_mask.png', binary_mask * 255)


True

In [6]:
def binary_mask_to_xy(binary_mask):
    # Find the coordinates of white pixels (value 1)
    white_points = np.argwhere(binary_mask == 1)

    # Create an array to hold the xy coordinates
    xy_array = np.zeros((white_points.shape[0], 2), dtype=int)

    # Fill the array with x, y coordinates
    xy_array[:, 0] = white_points[:, 1]  # x coordinate (column)
    xy_array[:, 1] = white_points[:, 0]  # y coordinate (row)

    return xy_array

# Assuming you have already created your binary mask
# binary_mask = create_binary_mask(image)

# Convert the binary mask to xy coordinates
coordinates = binary_mask_to_xy(binary_mask)
print(coordinates.shape)
db = DBSCAN(eps = 5, min_samples = 60).fit(coordinates)
flags = db.labels_
print(flags.shape)

(13579, 2)
(13579,)


In [7]:
def remove_coordinates_by_integer_flags(coordinates, flags):
    """
    Removes coordinates from the first array based on -1 values in the second array
    and appends the flags as a third column.

    Args:
        coordinates: A NumPy array of 2D coordinates (shape: Nx2).
        flags: A NumPy array of integers (shape: N).

    Returns:
        A new NumPy array containing the filtered coordinates with flags as the third column.
    """
    # Append the flags as a third column to the coordinates array
    coordinates_with_flags = np.column_stack((coordinates, flags))

    # Create a mask where True indicates coordinates to keep
    masks = flags != -1

    # Return the filtered coordinates with flags
    return coordinates_with_flags[masks]

filtered_coordinates = remove_coordinates_by_integer_flags(coordinates, flags)
# Print the filtered coordinates
print(filtered_coordinates.shape)
np.set_printoptions(threshold=np.inf)
print(filtered_coordinates)

(6569, 3)
[[ 98  47   0]
 [102  47   0]
 [ 96  48   0]
 [ 97  48   0]
 [ 98  48   0]
 [101  48   0]
 [102  48   0]
 [ 95  49   0]
 [ 96  49   0]
 [ 97  49   0]
 [ 98  49   0]
 [ 99  49   0]
 [100  49   0]
 [103  49   0]
 [106  49   0]
 [107  49   0]
 [109  49   0]
 [110  49   0]
 [ 95  50   0]
 [ 96  50   0]
 [ 97  50   0]
 [ 98  50   0]
 [ 99  50   0]
 [100  50   0]
 [101  50   0]
 [102  50   0]
 [103  50   0]
 [104  50   0]
 [105  50   0]
 [106  50   0]
 [107  50   0]
 [108  50   0]
 [109  50   0]
 [110  50   0]
 [111  50   0]
 [112  50   0]
 [114  50   0]
 [115  50   0]
 [ 94  51   0]
 [ 95  51   0]
 [ 96  51   0]
 [ 97  51   0]
 [ 98  51   0]
 [ 99  51   0]
 [100  51   0]
 [101  51   0]
 [102  51   0]
 [103  51   0]
 [104  51   0]
 [105  51   0]
 [106  51   0]
 [107  51   0]
 [108  51   0]
 [109  51   0]
 [110  51   0]
 [111  51   0]
 [112  51   0]
 [113  51   0]
 [114  51   0]
 [115  51   0]
 [ 94  52   0]
 [ 95  52   0]
 [ 96  52   0]
 [ 97  52   0]
 [ 98  52   0]
 [ 99  52   0]


In [8]:
# Create a 256x256 black image
image_size = (256, 256)
image = np.zeros(image_size, dtype=np.uint8)
# Set pixels at the given coordinates to white
for i in range(filtered_coordinates.shape[0]):
    x, y, c = filtered_coordinates[i][0], filtered_coordinates[i][1], filtered_coordinates[i][2]
    print(x, " ", y)
    image[y, x] = 255  # Note that image indexing is (row, column)
print(image.shape)
print(image[0])
# Save or display the image using OpenCV
cv2.imshow('Final', image)
cv2.waitKey(0)  # Wait for a key press to close the image window
cv2.destroyAllWindows()

98   47
102   47
96   48
97   48
98   48
101   48
102   48
95   49
96   49
97   49
98   49
99   49
100   49
103   49
106   49
107   49
109   49
110   49
95   50
96   50
97   50
98   50
99   50
100   50
101   50
102   50
103   50
104   50
105   50
106   50
107   50
108   50
109   50
110   50
111   50
112   50
114   50
115   50
94   51
95   51
96   51
97   51
98   51
99   51
100   51
101   51
102   51
103   51
104   51
105   51
106   51
107   51
108   51
109   51
110   51
111   51
112   51
113   51
114   51
115   51
94   52
95   52
96   52
97   52
98   52
99   52
100   52
101   52
102   52
103   52
104   52
105   52
106   52
107   52
108   52
109   52
110   52
111   52
112   52
113   52
114   52
115   52
116   52
117   52
120   52
121   52
97   53
98   53
99   53
100   53
101   53
102   53
103   53
104   53
105   53
106   53
107   53
108   53
109   53
110   53
111   53
112   53
113   53
115   53
116   53
117   53
118   53
119   53
120   53
121   53
122   53
129   53
98   54
99   54
100  

In [19]:
def imclose_binary(img, kernel_size):
    # Create a kernel
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Perform closing operation
    closed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    return closed
def imfill(img):
    # Create a mask larger than the image
    h, w = img.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from the corner
    filled_image = img.copy()
    cv2.floodFill(filled_image, mask, (0,0), 255)

    # Invert the filled image
    filled_image_inv = cv2.bitwise_not(filled_image)

    # Combine the filled image with the original image
    result = img | filled_image_inv

    return result
cv2.imshow('image', image)
cv2.waitKey(0)  # Wait for a key press to close the image window
cv2.destroyAllWindows()
"""filled_image = imfill(image)
cv2.imshow('filled_image', filled_image)
cv2.waitKey(0)  # Wait for a key press to close the image window
cv2.destroyAllWindows()"""
closed_image = imclose_binary(image, 6)
cv2.imshow('Closed_filled_image', closed_image)
cv2.waitKey(0)  # Wait for a key press to close the image window
cv2.destroyAllWindows()